# GIMP Subsetter Notebook

## Purpose

This notebook allows users to download subsets of [GIMP](https://nsidc.org/data/measures/gimp) image ([NSIDC-0723](https://nsidc.org/data/nsidc-0723)) and velocity ([NSIDC-481](https://nsidc.org/data/nsidc-0481), [0725](https://nsidc.org/data/nsidc-0725), [0727](https://nsidc.org/data/nsidc-0727), [0731](https://nsidc.org/data/nsidc-0731)) data. For the Sentinel based velocity mosaics (0725, 0727, 0731), the user can select a box on a map and select which components to be downloaded (vv, vx, vy, ex, ey, dT) and saved to a netCDF file. Once the download is complete, users can [explore](#Visualizing-the-Data) the data by interactively selecting points that are plotted as time series. In the case of the TSX products (NSDIC-0481), given their small size, the full product "box" is downloaded. Because of the sparse nature of these boxes, only the the products associated with a single box can be downloaded at a time.

While this notebook relies python, it is designed to be usable with no knowledge of python. In most cases, the default behavior will acomplish what most users want and customization can be carried out via minor tweaks of the parameters in the examples. To accomplish this goal, much of the actual python code is in [gimpfunc](https://github.com/fastice/gimpfunc) libary rather than the notebook itself. 

## Setup

There are several python packages that need to be installed to execute this notebook and potentially some jupyter lab/notebook extensions. Please follow the procedures outlined in the [**NSIDCLoginNotebook**](https://github.com/fastice/GIMPNotebooks/blob/master/NSIDCLoginNotebook.ipynb). The instructions there assume an Anaconda (conda) install but should translate well to pip or other package managers. The following cell will load all of the necessary packages. If errors occur, make sure all of the packages in the NSIDCLoginNotebook are installed. The login notebook will also setup the user's NASA Earthdata credentials that are required to successfully download data through this notebook.

Execute the rest of the cells in this section to setup the software.

In [ ]:
import gimpfunc as gimp
import panel as pn
pn.extension()
import rioxarray
import xarray as xr
import holoviews as hv
import os
import dask
import pandas as pd
from dask.diagnostics import ProgressBar
import param
#from dask.distributed import Client, performance_report
#from datetime import datetime

These files should have been created using **NSIDCLoginNotebook**. This step can avoided if the environment variables have been set elsewhere.

In [ ]:
env = dict(GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/.gimp_download_cookiejar.txt'),
            GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/.gimp_download_cookiejar.txt'))
os.environ.update(env)

These routine are used below to extract the product ID (e.g., NSIDC-0725) and type (e.g., image, sigma0) .

In [ ]:
def productID(cog):
    return cog.split('/')[-3].split('.')[0]

def boxName(cog):
    if 'NSIDC-0481' not in cog:
        return ''
    return '.' + cog.split('/')[-1].split('_')[1]

def productType(cog):
    return cog.split('/')[-1].split('_')[-3]

## Preliminary Setup

The first step is to decide the directory where the subsetted results will be saved. If the directory is not present, it will be created (note the directory where it will be created must exist). The final product filename can be customized [below](#set_filename). 

In [ ]:
subsetPath = '../Subsets'  # Modify as needed
if not os.path.exists(subsetPath):
    os.mkdir(subsetPath)  # Will fail if the directories above don't exist

## Find Data

The first step is to locate the products of interest, which is done using the same search tool that is used for [**qgisRemoteNotebook**](https://github.com/fastice/GIMPNotebooks/blob/master/qgisRemoteNotebook.ipynb), but with some key differences. Specifically, to avoid mixing data products of different sizes and resolutions, only a single product type (e.g., NSIDC-0723 *image* mosaics) can be searched for and retrieved at a time. For velocity products, the desired bands (*vv, vy, vx, ex, ey*, and *dT*) can be selected at later [stage](#Preload-Data-and-Select-Bands). Unlike the [**qgisRemoteNotebook**](https://github.com/fastice/GIMPNotebooks/blob/master/qgisRemoteNotebook.ipynb) search, TSX (NSIDC-0481) boxes are specified by name (e.g., *W69.10N*) rather than by a bounding box. A map of the box locations included in the NSIDC-0481 [User Guide](https://nsidc.org/data/nsidc-0481/versions/3).

To carry out a seearch, runthe next cell to bring up the search panel and perform a search. Once a search has completed and the desired products are listed, proceed to the [next steps](#Spatial-Subsetting). Once the results have been processed and downloaded, a new search can be peformed to find and download additional data. 

In [ ]:
myUrls = gimp.cmrUrls(mode='subsetter')  # Subsetter mode is required for subsetting.
myUrls.view()

## Subsetting Basics

Examples are included below for [Velocity Mosaics](#Velocity-Mosaics-(NSIDC-0725,-0727,-and-0731)), [Individual Glaciers](#Indidvidual-Glaciers-(NSIDC-0481)), and [Image Mosaics](#Image-Mosaics). Before jumping to one one of these steps, its important to describe the data formats and to provide some background on the tools used.

### NetCDF and Xarray

The subsetted data are downloaded to NetCDF [Network Common Data Format](https://www.unidata.ucar.edu/software/netcdf/) files, which allows storage of data with multiple layers coincident with the meta data describing these layers. For GIMP data sets it is particularly useful because multiple maps (hundreds or more) in a time series can be stored in a single file and easily accessed. By contrast, if the data are stored in individual files, potentially hundreds of files need to be opened to access a single point in space and time.

This and subsequent notebooks make extensive use of [xarray](http://xarray.pydata.org/en/stable/why-xarray.html), which is a python library that is especially well suited to working with NetCDF files. In particular, it can read NetCDF files into well-labeled arrays along with meta-data structures. Here we take also advantage of [rioxarray](https://corteva.github.io/rioxarray/stable/), which builds on xarray to add the ability append coordinate reference system [CRS](https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems) information. It also adds capability from the [rasterio](https://rasterio.readthedocs.io/en/latest/) library for working with geospatial imagery. These programs also utilize [dask](https://dask.org/) to perform parallel operations, which can greatly speed data access. The main focus of this notebook is on subsetting the data rather than working with the downloaded result. A subsequent notebook will examples of working with xarray data in greater detail. That said, a tool for visualizing the data for basic inspection is included [below](#Visualizing-the-Data).

The data sets stored as xarrays are organized as 4-D arrays indexed by **time** ([np.datetime64](https://numpy.org/doc/stable/reference/arrays.datetime.html)), **component** (e.g., 'vx', 'vy', 'image'), and **x** and **y** [EPSG 3413](https://epsg.io/3413)-projection coordinates. The time range is selected as part of the search described above. The appropriate components can be selected [below](#Visualizing-the-Data). For all but NISDC-0481, the spatial subsetting is performed as the next step. 

## Spatial Subsetting

The area to be subsetted is specified as a python dictionary with minimum and maximum *x* and *y* values. Before selecting a box consider the size. For example, with no compression once the data are read, the full NISDC-0723 image data set is >1.700 TB (just for one band). The main benefit of this notebook is that a particular region of interest can be extracted without requiring that the whole data set be downloaded, greatly reducing the volume of data to be downloaded and stored. There are two methods for selecting the region of interest. Before the final subsetting, the user can review the size of the product and iterated as needed to obtain a reasonable sized data set. While the region of interest can be selected, all products are downloaded at their original grid spacing.

### Method 1: Manual Selection

The coordinates for bounding box, `bbox`, can be manually entered by modifying the cell below with the desired values. Even if not using interactive [selection](#Method-2:-Interactive-Selection), running that step displays the manually selected box coordinates on radar map of Greenland. Note by default, coordinates are rounded to the nearest kilometer. This behavior can be modified [below](#rounding_info).

In [ ]:
bbox = {'minx': -243500, 'miny': -2295000, 'maxx': -149000, 'maxy': -2255000}  # Modifiy values as needed
boxPicker = gimp.boxPicker(bbox=bbox)  # Create a map for possible viewing

If a box was saved as part of a prior [download](#Subset-and-Download-Data), it can be reloaded here by uncommenting and modifying the cell below.

In [ ]:
# boxPicker = gimp.boxPicker(boxFile='changetoboxname.yaml')

### Method 2: Interactive Selection

Run the next the tool below to select the bounding box (or modify a manually selected box), which will display a SAR image map. Depending on network speed, it could take a few seconds to a minute to load the basemap. Use the box tool in the plot menu to select a region of interest. Then proceed to the next [step](#Preload-Data-and-Select-Bands).

In [ ]:
if 'boxPicker' not in locals(): # Only create if not defined above
    boxPicker = gimp.boxPicker()
boxPicker.plotMap()

Once the desired box has been selected, proceed to appropriate section for the selected product type:  [Velocity Mosaics](#Velocity-Mosaics-(NSIDC-0725,-0727,-and-0731)), [Individual Glaciers](#Indidvidual-Glaciers-(NSIDC-0481)), and [Image Mosaics](#Image-Mosaics-(NSIDC-0723)).

## Preload Data and Select Bands

The cells in this section read the COG headers and create an xarray inside a GIMPSubsetter object. The actual data are not downloaded at this stage, but the xarray will read the header data of each product so it can efficiently access the data during later downloads. The bands (e.g., vx, vy) can be selected at this stage.

Note the product-specific cells will only run if the appropriate data type was selected. As a result, if everything else has been configured correctly, the rest of the notebook can be executed and it will skip unecessary cells.

### Velocity Mosaics (NSIDC 0725, 0727, and 0731)

If velocity mosaic products were selected, this cell will read the cog headers and create an xarray inside a `GIMPSubsetter` object. The `bands` list below can be edited to select which bands will be downloaded (see [user guide](https://nsidc.org/data/nsidc-0725) for band info).

In [ ]:
if productID(myUrls.getCogs()[0]) in ['NSIDC-0725', 'NSIDC-0727', 'NSIDC-0731']:
    ProgressBar().register()
     # Edit to add or remove bands
    products = gimp.GIMPSubsetter(urls=myUrls.getCogs(), 
                                  bands=['vv', 'vx', 'vy', 'ex', 'ey', 'dT']) # vx,vy,vy, ex, ey, dT
    products.loadDataArray()  # Open COGS and read meta data.
else:
    print('No Velocity Mosaics data selected')

If this block ran successfully, the data are now ready to be [downloaded](#Subset-and-Download-Data).

### Indidvidual Glaciers (NSIDC-0481)

If individual glacier products were selected, this cell will read the cog headers and create an xarray inside a GIMPSubsetter object. The bands list below can be edited to select which bands will be downloaded (see [user guide](https://nsidc.org/data/nsidc-0481/versions/3) for band info).

In [ ]:
if productID(myUrls.getCogs()[0]) in ['NSIDC-0481']:
    ProgressBar().register()
     # Edit to add or remove bands
    products = gimp.GIMPSubsetter(urls=myUrls.getCogs(), 
                                          bands=['vv', 'vx', 'vy', 'ex', 'ey']) # vx,vy,vy, ex, ey
    products.loadDataArray()  # Open COGS and read meta data.
else:
    print('No Individual Glacier data selected')

If this block ran successfully, the data are now ready to be [downloaded](#Subset-and-Download-Data).

### Image Mosaics (NSIDC-0723)

The procedure for image mosaiacs is similar to that for the velocity data. The major exception is that only a single band can be downloaded at a time to avoid downloading bands with different sizes. This single band is defined through the product filter in [search panel](#Find-Data). Repeat as necessary to create download other bands.

In [ ]:
if productID(myUrls.getCogs()[0]) in ['NSIDC-0723']:
    ProgressBar().register()
     # Edit to add or remove bands
    products = gimp.GIMPSubsetter(urls=myUrls.getCogs(), 
                                          bands=[productType(myUrls.getCogs()[0])]) # band auto selected.
    products.loadDataArray()  # Open COGS and read meta data.
else:
    print('No Image Mosaic data selected')

If this block ran successfully, the data are now ready to be [downloaded](#Subset-and-Download-Data).

## Subset and Download Data

Before applying the final subset, its useful to examine the size of the full data array. If the `loadDataArray` step was sucessful, this next cell will provide details on the size and organization of the full xarray (prior to any download).

In [ ]:
products.DA # Add ; at the end to suppress output

For products other than those in NISDIC-0481, this next step will clip the data set to the bounding box created [above](#Spatial-Subsetting) and display the organization of the resulting subset. 

The box coordinates are rounded to the nearest km, which can be altered by changing the value of `decimals` below ([see numpy.around](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.around.html#numpy.around)). <a id='rounding_info'></a>

In [ ]:
if productID(myUrls.getCogs()[0]) not in ['NSIDC-0481']:  # Anything but a 481
    products.subSet(boxPicker.boxBounds(decimals=-3))  # -3 rounds to 1km, -2 to 100m...
else:
    print('NSIDC-0481 - so entire data set will be saved')
products.subset # Add ; at the end to suppress output

If the subset product seems a reasonable size, execute the next statement to save the data. If not, repeat [search](#Find-Data) or choose a different [bounding box](#Spatial-Subsetting). The next cell is used to specify the name, which will default to *subsetPath/prefix.NSIDC-0XXX.nc*. For NSIDC-0481 products, the box name (e.g., W69.1N) will be appended. This cell can be modified to override the default name as needed. In particular, the prefix can be updated to create names unique to each search. <a id='set_filename'></a>

In [ ]:
prefix = 'GIMPSubset'  # Rename as needed
# Add path defined above and append with productID (e.g., NSIDC-0723)
firstCog = myUrls.getCogs()[0]
subsetFile = f'{subsetPath}/{prefix}.{productID(firstCog)}{boxName(firstCog)}.nc'
if productID(myUrls.getCogs()[0]) not in ['NSIDC-0481']:  # Anything but a 481
    boxPicker.saveBox(subsetFile.replace('nc', 'yaml'))  # Save box for non 481 products

The data are now ready for download, which will be accomplished by running the next cell. Note, this step uses *dask* to improve download speed with multiple parallel streams ('workers'). While the NISDC limit is 15, if errors can cause timeouts so that the job may crash, particularly with some of the older COG formats (all products should be upated by late Spring 2021). Too many workers may also saturate the available network bandwidth. With 4 workers, 562 W69.1N (aka Jakobshavn) products with all components (6.88GB) downloaded sucessfully in 13.3 minutes through the University of Washington network.

In [ ]:
if productID(myUrls.getCogs()[0]) not in ['NSIDC-0481']:
    products.subSetToNetCDF(subsetFile, numWorkers=4)  # Use lower number if crashes, bump up <15 to increase download speed.
else:
    products.saveAll(subsetFile, numWorkers=4)

## Visualizing the Data

This section is designed for quick exploration of the data to determine whether the data were satisfactorily downloaded. The next step will open the netCDF file as an xarray and display a summary of its organization.

In [ ]:
try: # This action will clear any prexisting cache for a file with the same name
    myData.close()
except Exception:
    pass
# Now open data
myData = xr.open_dataset(subsetFile)
myData  # Add ; at the end to suppress output

The next cell allows the user to select points on the map and click to product a timeseries plot of one of the components in the data set. Change the `component` and rerun to visualize other components.

In [ ]:
component = str(myData.component[0].values)  # Picks first component- manually override as needed.
# component = 'vv' # Uncomment and modify to override default. 
points = gimp.pointInspector(myData, component=component)
points.view(component=component, mapTitle=f'{productID(myUrls.getCogs()[0])} {component}', plotTitle=f'{productID(myUrls.getCogs()[0])} {component}')  # Change 

This tool is largely for inspection, rather than producing publication ready plots. The following keywords can be updated and passed to `points.view()`. Note the defaults are data type dependent.

In [ ]:
print(points.imgOptions)
print(points.plotOptions)

In [ ]:
Execute this cell to get a listing of downloaded products

In [ ]:
!ls -l {subsetPath}

This notebook is now complete. If everything ran sucessfully the data is ready for analysis in other notebooks or applications.